In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
data = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')

In [3]:
data.fillna({
    'Episode_Length_minutes': data['Episode_Length_minutes'].mean(),
    'Guest_Popularity_percentage': data['Guest_Popularity_percentage'].mean(),
    'Number_of_Ads': data['Number_of_Ads'].mean()
}, inplace=True)

In [4]:
data.drop('id', axis = 1, inplace = True)

In [5]:
data

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,Mystery Matters,Episode 98,64.504738,True Crime,74.81,Thursday,Night,52.236449,0.0,Positive,31.41998
1,Joke Junction,Episode 26,119.800000,Comedy,66.95,Saturday,Afternoon,75.950000,2.0,Negative,88.01241
2,Study Sessions,Episode 16,73.900000,Education,69.97,Tuesday,Evening,8.970000,0.0,Negative,44.92531
3,Digital Digest,Episode 45,67.170000,Technology,57.22,Monday,Morning,78.700000,2.0,Positive,46.27824
4,Mind & Body,Episode 86,110.510000,Health,80.07,Monday,Afternoon,58.680000,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...
749995,Learning Lab,Episode 25,75.660000,Education,69.36,Saturday,Morning,52.236449,0.0,Negative,56.87058
749996,Business Briefs,Episode 21,75.750000,Business,35.21,Saturday,Night,52.236449,2.0,Neutral,45.46242
749997,Lifestyle Lounge,Episode 51,30.980000,Lifestyle,78.58,Thursday,Morning,84.890000,0.0,Negative,15.26000
749998,Style Guide,Episode 47,108.980000,Lifestyle,45.39,Thursday,Morning,93.270000,0.0,Negative,100.72939


In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [7]:
data

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,Mystery Matters,Episode 98,64.504738,True Crime,74.81,Thursday,Night,52.236449,0.0,Positive,31.41998
1,Joke Junction,Episode 26,119.800000,Comedy,66.95,Saturday,Afternoon,75.950000,2.0,Negative,88.01241
2,Study Sessions,Episode 16,73.900000,Education,69.97,Tuesday,Evening,8.970000,0.0,Negative,44.92531
3,Digital Digest,Episode 45,67.170000,Technology,57.22,Monday,Morning,78.700000,2.0,Positive,46.27824
4,Mind & Body,Episode 86,110.510000,Health,80.07,Monday,Afternoon,58.680000,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...
749995,Learning Lab,Episode 25,75.660000,Education,69.36,Saturday,Morning,52.236449,0.0,Negative,56.87058
749996,Business Briefs,Episode 21,75.750000,Business,35.21,Saturday,Night,52.236449,2.0,Neutral,45.46242
749997,Lifestyle Lounge,Episode 51,30.980000,Lifestyle,78.58,Thursday,Morning,84.890000,0.0,Negative,15.26000
749998,Style Guide,Episode 47,108.980000,Lifestyle,45.39,Thursday,Morning,93.270000,0.0,Negative,100.72939


In [8]:
x = data.drop('Listening_Time_minutes', axis=1)

In [9]:
y = data['Listening_Time_minutes']

In [10]:
# Define categorical features
categorical_features = ['Podcast_Name', 'Episode_Title', 'Genre', 
                        'Publication_Day', 'Publication_Time', 'Number_of_Ads', 'Episode_Sentiment']

# Initialize OneHotEncoder with `handle_unknown='ignore'`
one_hot = OneHotEncoder(handle_unknown='ignore')  

# Create transformer & fit it on training data ONLY
transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)], remainder='passthrough')
transformed_data = transformer.fit_transform(x)  # Fit only on training data
transformed_data = transformed_data.toarray()
transformed_data = pd.DataFrame(transformed_data)

In [11]:
transformed_data

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64.504738,74.81,52.236449
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,119.800000,66.95,75.950000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,73.900000,69.97,8.970000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,67.170000,57.22,78.700000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,110.510000,80.07,58.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,75.660000,69.36,52.236449
749996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,75.750000,35.21,52.236449
749997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30.980000,78.58,84.890000
749998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,108.980000,45.39,93.270000


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [14]:
# x_train

In [15]:
# x_test

In [16]:
# from sklearn.ensemble import RandomForestRegressor

In [17]:
# model = RandomForestRegressor()

In [18]:
# model.fit(x_train, y_train)

In [19]:
# model.score(x_test, y_test)

In [20]:
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=7)
model.fit(transformed_data, y)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [21]:
# model.score(x_test, y_test)
# dsfdfb

In [22]:
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')

In [23]:
test

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...
249995,999995,Mind & Body,Episode 100,21.05,Health,65.77,Saturday,Evening,96.40,3.0,Negative
249996,999996,Joke Junction,Episode 85,85.50,Comedy,41.47,Saturday,Night,30.52,2.0,Negative
249997,999997,Joke Junction,Episode 63,12.11,Comedy,25.92,Thursday,Evening,73.69,1.0,Neutral
249998,999998,Market Masters,Episode 46,113.46,Business,43.47,Friday,Night,93.59,3.0,Positive


In [24]:
x = test.drop('id', axis=1)


In [25]:
x.fillna({
    'Episode_Length_minutes': x['Episode_Length_minutes'].mean(),
    'Guest_Popularity_percentage': x['Guest_Popularity_percentage'].mean(),
    'Number_of_Ads': x['Number_of_Ads'].mean()
}, inplace=True)

In [26]:
x

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.330000,1.0,Neutral
1,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,52.192796,0.0,Neutral
2,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.510000,0.0,Positive
3,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.750000,2.0,Positive
4,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.300000,2.0,Neutral
...,...,...,...,...,...,...,...,...,...,...
249995,Mind & Body,Episode 100,21.05,Health,65.77,Saturday,Evening,96.400000,3.0,Negative
249996,Joke Junction,Episode 85,85.50,Comedy,41.47,Saturday,Night,30.520000,2.0,Negative
249997,Joke Junction,Episode 63,12.11,Comedy,25.92,Thursday,Evening,73.690000,1.0,Neutral
249998,Market Masters,Episode 46,113.46,Business,43.47,Friday,Night,93.590000,3.0,Positive


In [27]:
from sklearn.impute import SimpleImputer

In [28]:
test_ids = test['id']

In [29]:
x.dtypes

Podcast_Name                    object
Episode_Title                   object
Episode_Length_minutes         float64
Genre                           object
Host_Popularity_percentage     float64
Publication_Day                 object
Publication_Time                object
Guest_Popularity_percentage    float64
Number_of_Ads                  float64
Episode_Sentiment               object
dtype: object

In [30]:
x.isna().sum()

Podcast_Name                   0
Episode_Title                  0
Episode_Length_minutes         0
Genre                          0
Host_Popularity_percentage     0
Publication_Day                0
Publication_Time               0
Guest_Popularity_percentage    0
Number_of_Ads                  0
Episode_Sentiment              0
dtype: int64

In [31]:
transformed_x_test = transformer.transform(x)  # ⚠️ Use `transform()`, NOT `fit_transform()`


In [32]:
transformed_x_test = transformed_x_test.toarray()
transformed_x_test = pd.DataFrame(transformed_x_test)

In [33]:
transformed_x_test

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,78.96,38.11,53.330000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27.87,71.29,52.192796
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,69.10,67.89,97.510000
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,115.39,23.40,51.750000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,72.32,58.10,11.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,21.05,65.77,96.400000
249996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,85.50,41.47,30.520000
249997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.11,25.92,73.690000
249998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,113.46,43.47,93.590000


In [34]:
# predictions_xgb = model.predict(transformed_x_test)

In [35]:
# submission = pd.DataFrame({
#     'id': range(750000, 750000 + len(predictions)),  # Ensures it matches the number of predictions
#     'listening_time_predict': predictions
# })

In [36]:
# submission.to_csv("submission.csv", index=False)

In [37]:
# from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import Ridge
# from sklearn.svm import SVR
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor

In [38]:
# base_models = [
#     ('xgb', XGBRegressor(n_estimators=300, max_depth=8, learning_rate=0.05, subsample=0.8, n_jobs=-1)),
#     ('lgbm', LGBMRegressor(n_estimators=300, learning_rate=0.05, max_depth=-1, subsample=0.8, n_jobs=-1)),
#     ('catboost', CatBoostRegressor(verbose=0, iterations=300, learning_rate=0.05, depth=8))
# ]

In [39]:
# meta_learner = Ridge(alpha=1.0)

In [40]:
# ensemble_model = StackingRegressor(
#     estimators=base_models,
#     final_estimator=meta_learner,
#     n_jobs=-1,
#     passthrough=True  # optional: passes original features to meta-learner
# )

In [41]:
# ensemble_model.fit(transformed_data, y)

In [42]:
# predictions_ensemble = ensemble_model.predict(transformed_x_test)

In [43]:
# len(predictions_ensemble)

In [44]:
# submission = pd.DataFrame({
#     'id': range(750000, 750000 + len(predictions_ensemble)),  # Ensures it matches the number of predictions
#     'listening_time_predict': predictions_ensemble
# })

In [45]:
# submission.to_csv("submission.csv", index=False)

In [47]:
pip install ngboost


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 8.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 91.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 7.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=7b3e35f430a82a68bac51f975ba182b053b73d15c2c372587cfe35eed29f81c7
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1

In [48]:
from ngboost import NGBRegressor
from ngboost.distns import Normal
from ngboost.scores import MLE
from sklearn.metrics import mean_squared_error
import numpy as np


In [50]:
ngb = NGBRegressor(
    Dist=Normal,
    Score=MLE,  # 👈 No parentheses here
    n_estimators=500,
    learning_rate=0.03,
    natural_gradient=True,
    verbose=True
)

In [52]:
transformed_data

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64.504738,74.81,52.236449
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,119.800000,66.95,75.950000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,73.900000,69.97,8.970000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,67.170000,57.22,78.700000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,110.510000,80.07,58.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,75.660000,69.36,52.236449
749996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,75.750000,35.21,52.236449
749997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30.980000,78.58,84.890000
749998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,108.980000,45.39,93.270000


In [53]:
pip install scikit-learn==1.2.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 58.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
ngboost 0.5.5 requires scikit-learn<2.0,>=1.6, but you have scikit-learn 1.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [54]:
from ngboost import NGBRegressor
from ngboost.distns import Normal
from ngboost.scores import MLE
from sklearn.metrics import mean_squared_error
import numpy as np

ngb = NGBRegressor(
    Dist=Normal,
    Score=MLE,
    n_estimators=500,
    learning_rate=0.03,
    natural_gradient=True,
    verbose=True
)

In [55]:
ngb.fit(transformed_data, y)

TypeError: check_X_y() got an unexpected keyword argument 'ensure_all_finite'